In [ ]:
# Task 2.3.1: Use glob to create a list that contains the filenames for all the Buenos Aires real estate CSV files in the data directory. 
# Assign this list to the variable name files.
from glob import glob
files = glob("./data/buenos-aires-real-estate-[1-5].csv")
files

In [ ]:
# Task 2.3.2: Use your wrangle function in a for loop to create a list named frames. 
# The list should the cleaned DataFrames created from the CSV filenames your collected in files.
frames = []
for file in files:
    df = wrangle(file)
    frames.append(df)

In [ ]:
# Task 2.3.3: Use pd.concat to concatenate the items in frames into a single DataFrame df. 
# Make sure you set the ignore_index argument to True.

df = pd.concat(frames, ignore_index = True)
print(df.info())
df.head()

In [ ]:
# Task 2.3.4: Modify your wrangle function to create a new feature "neighborhood". 
# You can find the neighborhood for each property in the "place_with_parent_names" column. 
# For example, a property with the place name "|Argentina|Capital Federal|Palermo|" is located in the neighborhood is "Palermo". 
# Also, your function should drop the "place_with_parent_names" column.

def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)

    # Extract neighborhood
    df["neighborhood"] = df["place_with_parent_names"].str.split("|",expand=True)[3]
    df.drop(columns="place_with_parent_names",inplace=True)

    return df

In [ ]:
# Task 2.3.5: Create your feature matrix X_train and target vector y_train. 
# X_train should contain one feature: "neighborhood". Your target is "price_aprox_usd". 

target = "price_aprox_usd"
features = ["neighborhood"]
y_train = df[target]
X_train = df[features]